In [ ]:
import os
import glob

import numpy as np
import pandas as pd

# Don't install PyNIO into any environment that also has xarray
import xarray as xr

import metpy.calc as mpcalc
from metpy.units import units

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.collections import LineCollection
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import matplotlib.patheffects as pe
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import math
import dask.dataframe as dd
from scipy.spatial import Delaunay

from uviz.utils.tools import find_TC_bbox
from uviz.datashader_tools.utils import datashader_wrapper
from uviz.plotting.utils import basin_bboxes, ssh_wsp, ssh_mslp, sshws_color, geog_features

mpl.rcParams['axes.labelsize'] = 12
mpl.rcParams['axes.titlesize'] = 14
mpl.rcParams['axes.titleweight'] = 'bold'

In [ ]:
def child_files(root_dir):
    # TODO: write in flag for init time (not necessary yet)
    
    # Daily averages
    h1_files = glob.glob(os.path.join(root_dir, '*.h1.*[!remap].nc'))
    
    # 6-hourly snapshots or 6-hourly averages (depends on variable)
    h2_files = glob.glob(os.path.join(child_3km, '*.h2.*[!remap].nc'))
    
    # 3-hourly snapshots
    h3_files = glob.glob(os.path.join(child_3km, '*.h3.*[!remap].nc'))
    
    # Hourly averages (for accumulation)
    h4_files = glob.glob(os.path.join(child_3km, '*.h4.*[!remap].nc'))
    
    # 15-minute surface winds, precip, etc.
    h5_files = os.listdir(os.path.join(root_dir, 'h5'))
    
    return h1_files, h2_files, h3_files, h4_files#, h5_files
    

In [ ]:
prototype_dir = r"/gpfs/group/cmz5202/default/cnd5285/synth_events"
storm_1279_dir = os.path.join(prototype_dir, "v0.1/VR28.NATL.EXT.CAM5.4CLM5.0.dtime900_storm_1279")
parent_28km = os.path.join(storm_1279_dir, '28km')
child_3km = os.path.join(storm_1279_dir, '3km')

In [ ]:
# h1 - daily averages
# h2 - 6-hourly snapshots (i.e., instantaneous at that time) or 6-hourly averages 
# (average of previous 6 hours, depends on variable, query metadata for information)
# h3 - 3-hourly snapshots
# h4 - hourly averages (for accumulation -- e.g., total precip (m) that fell in each hour = m/s * 3600s
# h5 - unique to the downscaled sims that is 15-minute surface winds, precip, etc. 

### Parent Sims

In [ ]:
h1p_files = glob.glob(os.path.join(parent_28km, '*.h1.*.nc'))
h2p_files = glob.glob(os.path.join(parent_28km, '*.h2.*.nc'))
h3p_files = glob.glob(os.path.join(parent_28km, '*.h3.*.nc'))
h4p_files = glob.glob(os.path.join(parent_28km, '*.h4.*.nc'))

# CAM mesh is the "ext" one, based on original model run. Other options are "ref" or "wat"
p_mesh = r"/gpfs/group/cmz5202/default/cnd5285/maps_and_grids/ne0np4natlanticext.ne30x4.g_scrip.nc"

In [ ]:
# Native grid files
parallel = True
h1pn_ds = xr.open_mfdataset([f for f in h1p_files if 'remap' not in f], parallel=parallel)
h2pn_ds = xr.open_mfdataset([f for f in h2p_files if 'remap' not in f], parallel=parallel)
h3pn_ds = xr.open_mfdataset([f for f in h3p_files if 'remap' not in f], parallel=parallel)
h4pn_ds = xr.open_mfdataset([f for f in h4p_files if 'remap' not in f], parallel=parallel)
p_mesh_ds = xr.open_dataset(p_mesh)

# Regridded files
#h1pr_ds = xr.open_mfdataset([f for f in h1p_files if 'remap' in f], parallel=parallel)
h2pr_ds = xr.open_mfdataset([f for f in h2p_files if 'remap' in f], parallel=parallel)
h3pr_ds = xr.open_mfdataset([f for f in h3p_files if 'remap' in f], parallel=parallel)
h4pr_ds = xr.open_mfdataset([f for f in h4p_files if 'remap' in f], parallel=parallel)

In [ ]:
h4pn_ds

In [ ]:
h3pr_ds

In [ ]:
p_mesh_ds

### Child Sims

In [ ]:
h1c_files = glob.glob(os.path.join(child_3km, '*.h1.*.nc'))
h2c_files = glob.glob(os.path.join(child_3km, '*.h2.*.nc'))
h3c_files = glob.glob(os.path.join(child_3km, '*.h3.*.nc'))
h4c_files = glob.glob(os.path.join(child_3km, '*.h4.*.nc'))
h5c_files = os.listdir(os.path.join(child_3km, 'h5'))

# Dug in the attributes for the mesh, TODO: make function that finds it automatically
c_mesh = "/gpfs/group/cmz5202/default/cnd5285/MPAS_3km/x20.835586.florida.init.CAM.nc"

In [ ]:
parallel=True
# Native grid files
h1cn_ds = xr.open_mfdataset([f for f in h1c_files if 'remap' not in f], parallel=parallel)
h2cn_ds = xr.open_mfdataset([f for f in h2c_files if 'remap' not in f], parallel=parallel)
h3cn_ds = xr.open_mfdataset([f for f in h3c_files if 'remap' not in f], parallel=parallel)
h4cn_ds = xr.open_mfdataset([f for f in h4c_files if 'remap' not in f], parallel=parallel)
#h5cn_ds = xr.open_mfdataset([f for f in h5c_files])
c_mesh_ds = xr.open_dataset(c_mesh, chunks={'nCells':10000, 'nVertices':100, 'nEdges':100})

# Regridded files
h1cr_ds = xr.open_mfdataset([f for f in h1c_files if 'remap' in f], parallel=parallel)
h2cr_ds = xr.open_mfdataset([f for f in h2c_files if 'remap' in f], parallel=parallel)
h3cr_ds = xr.open_mfdataset([f for f in h3c_files if 'remap' in f], parallel=parallel)
h4cr_ds = xr.open_mfdataset([f for f in h4c_files if 'remap' in f], parallel=parallel)

In [ ]:
h3pr_ds['WSP850'] = mpcalc.wind_speed(h3pr_ds['U850'], h3pr_ds['V850'])
h3pn_ds['WSP850'] = mpcalc.wind_speed(h3pn_ds['U850'], h3pn_ds['V850'])
h3cr_ds['WSP850'] = mpcalc.wind_speed(h3cr_ds['U850'], h3cr_ds['V850'])
h3cn_ds['WSP850'] = mpcalc.wind_speed(h3cn_ds['U850'], h3cn_ds['V850'])

In [ ]:
h2cr_ds

In [ ]:
h3cn_ds

In [ ]:
h4cn_ds

In [ ]:
c_mesh_ds

# Analysis

### Track Comparison

In [ ]:
def geog_features(ax, basin='north atlantic zoomed', resolution='10m'):
    
    lons, lats = basin_bboxes(basin)
    ax.set_extent([lons[0], lons[1], lats[0], lats[1]], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale(resolution), linewidth=0.5, edgecolor='#323232', zorder=3)
    #ax.add_feature(cfeature.BORDERS.with_scale(resolution), linewidth=0.5, edgecolor='#323232', zorder=3)
    ax.add_feature(cfeature.STATES.with_scale(resolution), linewidth=0.5, facecolor='#EBEBEB', edgecolor='#616161', zorder=2)
    ax.add_feature(cfeature.LAKES.with_scale(resolution), linewidth=0.5, facecolor='#e4f1fa', edgecolor='#616161', zorder=2)
    ax.add_feature(cfeature.OCEAN.with_scale(resolution), facecolor='#e4f1fa', edgecolor='face', zorder=1)

# def sshws_color(wsp, units='m/s'):
#     category = saffir_simpson(wsp, units)
    
#     if category == 'Tropical Depression':
#         return '#5EBAFF'
#     elif category == 'Tropical Storm':
#         return '#00FAF4'
#     elif category == 'Category 1':
#         return '#FFF795'
#     elif category == 'Category 2':
#         return '#FFD821'
#     elif category == 'Category 3':
#         return '#FF8F20'
#     elif category == 'Category 4':
#         return '#FF6060'
#     elif category == 'Category 5':
#         return '#C464D9'

### Minimum Sea Level Pressure and Maximum Wind Speed Trace Timeseries

In [ ]:
ew, sn = basin_bboxes('florida')

# This block extracts min SLP at each timestep + coordinates (pointwise, not gridded)
# Hourly
cn_subset = h3cn_ds.where((h3cn_ds['lon'] <= ew[1]) & (h3cn_ds['lon'] >= ew[0]) &
                          (h3cn_ds['lat'] <= sn[1]) & (h3cn_ds['lat'] >= sn[0]), drop=True)
cr_subset = h3cr_ds.sel(lon=slice(ew[0], ew[1]), lat=slice(sn[0], sn[1]))
cn_min_slps = cn_subset.PSL.sel(ncol=cn_subset.PSL.compute().argmin(dim='ncol'))
cr_min_slps = cr_subset.PSL.isel(cr_subset.PSL.compute().argmin(dim=('lon', 'lat')))

pn_subset = h3pn_ds.where((h3pn_ds['lon'] <= ew[1]) & (h3pn_ds['lon'] >= ew[0]) &
                          (h3pn_ds['lat'] <= sn[1]) & (h3pn_ds['lat'] >= sn[0]), drop=True)
pr_subset = h3pr_ds.sel(lon=slice(ew[0], ew[1]), lat=slice(sn[0], sn[1]))
pn_min_slps = pn_subset.PSL.sel(ncol=pn_subset.PSL.compute().argmin(dim='ncol'))
pr_min_slps = pr_subset.PSL.isel(pr_subset.PSL.compute().argmin(dim=('lon', 'lat')))

# This block extracts max WSP at each timestep + coordinates (pointwise, not gridded)
# Hourly
cn_max_wsps = cn_subset.U10.sel(ncol=cn_subset.U10.compute().argmax(dim='ncol'))
cn_max_wsps = cn_max_wsps.metpy.convert_units('mph')
cr_max_wsps = cr_subset.U10.isel(cr_subset.U10.compute().argmax(dim=('lon', 'lat')))
cr_max_wsps = cr_max_wsps.metpy.convert_units('mph')

pn_max_wsps = pn_subset.U10.sel(ncol=pn_subset.U10.compute().argmax(dim='ncol'))
pn_max_wsps = pn_max_wsps.metpy.convert_units('mph')
pr_max_wsps = pr_subset.U10.isel(pr_subset.U10.compute().argmax(dim=('lon', 'lat')))
pr_max_wsps = pr_max_wsps.metpy.convert_units('mph')

In [ ]:
np.linspace(0, len(h3cr_ds.time.values)*3, len(h3cr_ds.time.values)+1)

In [ ]:
fix, axs = plt.subplots(2, 1, dpi=200, figsize=(8, 12), sharex=True, layout='constrained')

#times = h3cr_ds.time.dt.strftime("%Y-%m-%d %H:%M").values
times = np.linspace(0, len(h3cr_ds.time.values)*3, len(h3cr_ds.time.values)+1)[1:]

lw = 1.5
titlesize = 24
labelsize = 18
ticksize=16

axs[0].plot(times, pn_min_slps.values/100, label='CAM5 (Native)', c='#024588')
axs[0].plot(times, pr_min_slps.values/100, label='CAM5 (Regridded)', c='#7a8fc2', ls='--', lw=lw)
axs[0].plot(times, cn_min_slps.values/100, label='MPAS (Native)', c='#FA03B2')
axs[0].plot(times, cr_min_slps.values/100, label='MPAS (Regridded)', c='#ff8bd1', ls='--', lw=lw)
axs[0].set_ylabel('Pressure [mb]', fontsize=labelsize)
axs[0].set_title('Minimum Sea Level Pressure', fontsize=titlesize)
axs[0].legend(fontsize=14, loc='upper right')

axs[1].plot(times, pn_max_wsps, label='CAM5 (Native)', c='#024588')
axs[1].plot(times, pr_max_wsps, label='CAM5 (Regridded)', c='#7a8fc2', ls='--', lw=lw)
axs[1].plot(times, cn_max_wsps, label='MPAS (Native)', c='#FA03B2')
axs[1].plot(times, cr_max_wsps, label='MPAS (Regridded)', c='#ff8bd1', ls='--', lw=lw)
axs[1].set_ylabel('Wind Speed [mph]', fontsize=labelsize)
axs[1].set_title('Maximum Surface Wind Speed', fontsize=titlesize)
axs[1].legend(fontsize=14, loc='lower right')
axs[1].set_xlabel('Hours Since Initialization', fontsize=labelsize)

for ax in axs.ravel():
    start, end = ax.get_xlim()
    #ax.xaxis.set_ticks(np.arange(start, end, 4))
    #plt.setp(ax.get_xticklabels(), ha='right', rotation=45)
    ax.tick_params(axis='both', which='major', labelsize=ticksize)

plt.show()
#fig.savefig('../figs/hourly_slp_wsp.png', bbox_inches='tight', dpi=200, transparent=False)

In [ ]:
ew, sn = basin_bboxes('florida')
wsp_var = 'WSP850'

# This block extracts max WSP at each timestep + coordinates (pointwise, not gridded)
# Hourly
cn_subset = h3cn_ds.where((h3cn_ds['lon'] <= ew[1]) & (h3cn_ds['lon'] >= ew[0]) &
                          (h3cn_ds['lat'] <= sn[1]) & (h3cn_ds['lat'] >= sn[0]), drop=True)
cr_subset = h3cr_ds.sel(lon=slice(ew[0], ew[1]), lat=slice(sn[0], sn[1]))
cn_max_wsps = cn_subset[wsp_var].sel(ncol=cn_subset[wsp_var].compute().argmax(dim='ncol')) * units('m/s')
cn_max_wsps = cn_max_wsps.metpy.convert_units('mph')
cr_max_wsps = cr_subset[wsp_var].isel(cr_subset[wsp_var].compute().argmax(dim=('lon', 'lat'))) * units('m/s')
cr_max_wsps = cr_max_wsps.metpy.convert_units('mph')

pn_subset = h3pn_ds.where((h3pn_ds['lon'] <= ew[1]) & (h3pn_ds['lon'] >= ew[0]) &
                          (h3pn_ds['lat'] <= sn[1]) & (h3pn_ds['lat'] >= sn[0]), drop=True)
pr_subset = h3pr_ds.sel(lon=slice(ew[0], ew[1]), lat=slice(sn[0], sn[1]))
pn_max_wsps = pn_subset[wsp_var].sel(ncol=pn_subset[wsp_var].compute().argmax(dim='ncol')) * units('m/s')
pn_max_wsps = pn_max_wsps.metpy.convert_units('mph')
pr_max_wsps = pr_subset[wsp_var].isel(pr_subset[wsp_var].compute().argmax(dim=('lon', 'lat'))) * units('m/s')
pr_max_wsps = pr_max_wsps.metpy.convert_units('mph')

fig, ax = plt.subplots(figsize=(12, 7), dpi=200)
times = h3cr_ds.time.dt.strftime("%Y-%m-%d %H:%M").values

ax.plot(times, pn_max_wsps, label='CAM (Native)', c='#024588')
ax.plot(times, pr_max_wsps, label='CAM (Regridded)', c='#7a8fc2', ls='--', lw=0.75)
ax.plot(times, cn_max_wsps, label='MPAS (Native)', c='#FA03B2')
ax.plot(times, cr_max_wsps, label='MPAS (Regridded)', c='#ff8bd1', ls='--', lw=0.75)

ax.set_ylabel('Wind Speed [mph]', fontsize=14)
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(start, end, 2))
plt.setp(ax.get_xticklabels(), ha='right', rotation=45)
ax.set_title('Maximum WSP (Hourly) - 850 mb', fontsize=16)
ax.legend(fontsize=12)
plt.show()

#fig.savefig('../figs/max_wsp_hourly.png', bbox_inches='tight', dpi=200, transparent=False)

### Track Evolution

In [ ]:
# According to Klotzbach et. al., 2020
# Modified for TS and TD designation using Kantha, 2006
# def ssh_mslp(slp, unit='pascal'):
#     pascals = ['pascal', 'Pascal', 'pa', 'Pa', 'pascals', 'Pascals']
#     hPa = ['mb', 'milibars', 'hPa', 'hectopascals', 'Hectopascals']
    
#     if unit in pascals:
#         slp = slp/100
    
#     if slp <= 925:
#         return 'Category 5'
#     elif 946 >= slp > 925:
#         return 'Category 4'
#     elif 960 >= slp > 946:
#         return 'Category 3'
#     elif 975 >= slp > 960:
#         return 'Category 2'
#     elif 990 >= slp > 975:
#         return 'Category 1'
#     elif 1000 > slp > 990:
#         return 'Tropical Storm'
#     elif slp >= 1000:
#         return 'Tropical Depression'

In [ ]:
pn_min_slps

In [ ]:
pn_min_slps.values

In [ ]:
te_times = pn_min_slps.time.dt.hour.isin([0, 6, 12, 18])
pn_lons = pn_subset.lon.sel(ncol=pn_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values
pn_lons

In [ ]:
pn_lats = pn_subset.lat.sel(ncol=pn_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values
pn_lats

In [ ]:
pn_slps = pn_min_slps.sel(time=te_times).values
pn_slps

In [ ]:
# Loads in track 1279, filters timespan window to model output window
times = h3cr_ds.time.dt.strftime("%Y-%m-%d %H:%M:%S").values
original_track = pd.read_csv('../tempest_extremes/storm_1279_track.csv', index_col=0)
filtered_ot = original_track[original_track['time'].isin(times)].reset_index(drop=True)
filtered_ot['ss_mslp'] = filtered_ot['slp'].apply(lambda x: ssh_mslp(x))

filtered_ot

In [ ]:
filtered_ot['wsp']*2.23694

In [ ]:
cn_wsps

In [ ]:
proj = ccrs.PlateCarree()
fig, axs = plt.subplots(1, 3, figsize=(17, 10), dpi=300, subplot_kw=dict(projection=proj), layout='constrained')
for ax in axs.ravel():
    geog_features(ax, 'florida')
    
title_fs = 18
plt_ms = 75

# Tempest Extremes Section
te_wsp = filtered_ot.wsp.values
te_lons = filtered_ot.lon.values
te_lats = filtered_ot.lat.values
ssh_wsp_cmap = [sshws_color(wsp, units='m/s') for wsp in te_wsp]
te_points = np.array([te_lons, te_lats]).T.reshape(-1, 1, 2)
te_segments = np.concatenate([te_points[:-1], te_points[1:]], axis=1)
lc = LineCollection(te_segments, colors='k', zorder=7, transform=proj, lw=0.5)
axs[0].add_collection(lc)
axs[0].scatter(te_lons, te_lats, c=ssh_wsp_cmap, zorder=8, edgecolors='k', lw=0.5, s=plt_ms, transform=proj)
axs[0].set_title('Original Storm_1279 Track', fontsize=title_fs)

# Parent Simulation
te_times = pn_max_wsps.time.dt.hour.isin([0, 6, 12, 18])
pn_wsps = pn_max_wsps.sel(time=te_times).values
pn_slps = pn_min_slps.sel(time=te_times).values
pn_lons = pn_subset.lon.sel(ncol=pn_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values
pn_lats = pn_subset.lat.sel(ncol=pn_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values

sshws_cmap = [sshws_color(wsp, units='mph') for wsp in pn_wsps]
pn_points = np.array([pn_lons, pn_lats]).T.reshape(-1, 1, 2)
pn_segments = np.concatenate([pn_points[:-1], pn_points[1:]], axis=1)
lc = LineCollection(pn_segments, colors='k', zorder=9, transform=proj, lw=0.5, ls='-')
axs[1].add_collection(lc)
axs[1].scatter(pn_lons, pn_lats, c=sshws_cmap, zorder=10, edgecolors='k', lw=0.5, s=plt_ms, transform=proj)
axs[1].set_title('Parent Simulation', fontsize=title_fs)

# Child Simulation
cn_wsps = cn_max_wsps.sel(time=te_times).values
cn_slps = cn_min_slps.sel(time=te_times).values
cn_lons = cn_subset.lon.sel(ncol=cn_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values
cn_lats = cn_subset.lat.sel(ncol=cn_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values

sshws_cmap = [sshws_color(wsp, units='mph') for wsp in cn_wsps]
cn_points = np.array([cn_lons, cn_lats]).T.reshape(-1, 1, 2)
cn_segments = np.concatenate([cn_points[:-1], cn_points[1:]], axis=1)
lc = LineCollection(cn_segments, colors='k', zorder=9, transform=proj, lw=0.5, ls='-')
axs[2].add_collection(lc)
axs[2].scatter(cn_lons, cn_lats, c=sshws_cmap, zorder=10, edgecolors='k', lw=0.5, s=plt_ms, transform=proj)
axs[2].set_title('Child Simulation', fontsize=title_fs)

# Marker properties
mew = 0.5     # marker edge width
mec = 'k'      # marker edge color
ms = 8         # marker size
#mfc           # marker face color

td = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Tropical Depression', mfc=sshws_color(35, 'mph'), color='k', lw=0.5, ls='-')
ts = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Tropical Storm', mfc=sshws_color(50, 'mph'), color='k', lw=0.5, ls='-')
c1 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 1', mfc=sshws_color(75, 'mph'), color='k', lw=0.5, ls='-')
c2 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 2', mfc=sshws_color(100, 'mph'), color='k', lw=0.5, ls='-')
c3 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 3', mfc=sshws_color(115, 'mph'), color='k', lw=0.5, ls='-')
c4 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 4', mfc=sshws_color(135, 'mph'), color='k', lw=0.5, ls='-')
c5 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 5', mfc=sshws_color(160, 'mph'), color='k', lw=0.5, ls='-')

miami_coords = (25.775163, -80.208615)
for ax in axs.ravel():
    ax.scatter(x=miami_coords[1]+360, y=miami_coords[0], zorder=6, marker='*', c='#00FF00', s=200, edgecolors='k', lw=0.5, transform=proj)
    l = ax.legend(handles = [c5, c4, c3, c2, c1, ts, td], loc='upper right', fontsize=12.5, shadow=False)
    l.set_zorder(1001)
    
#fig.suptitle('With SSHWS for Simulated Tracks', fontsize=20, fontweight='bold')

plt.show()

In [ ]:
proj = ccrs.PlateCarree()
fig, axs = plt.subplots(1, 3, figsize=(17, 10), dpi=200, subplot_kw=dict(projection=proj), layout='constrained')
for ax in axs.ravel():
    geog_features(ax, 'florida')
    
title_fs = 18
plt_ms = 75

# Tempest Extremes Section
te_mslp = filtered_ot.slp.values
te_lons = filtered_ot.lon.values
te_lats = filtered_ot.lat.values
ssh_mslp_cmap = [sshws_color(slp, units='pa') for slp in te_mslp]
te_points = np.array([te_lons, te_lats]).T.reshape(-1, 1, 2)
te_segments = np.concatenate([te_points[:-1], te_points[1:]], axis=1)
lc = LineCollection(te_segments, colors='k', zorder=7, transform=proj, lw=0.5)
axs[0].add_collection(lc)
axs[0].scatter(te_lons, te_lats, c=ssh_mslp_cmap, zorder=8, edgecolors='k', lw=0.35, s=plt_ms, transform=proj)
axs[0].set_title('Original Storm_1279 Track', fontsize=title_fs)

# Parent Simulation
te_times = pn_max_wsps.time.dt.hour.isin([0, 6, 12, 18])
pn_slps = pn_min_slps.sel(time=te_times).values
pn_lons = pn_subset.lon.sel(ncol=pn_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values
pn_lats = pn_subset.lat.sel(ncol=pn_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values

ssh_cmap = [sshws_color(slp, units='pa') for slp in pn_slps]
pn_points = np.array([pn_lons, pn_lats]).T.reshape(-1, 1, 2)
pn_segments = np.concatenate([pn_points[:-1], pn_points[1:]], axis=1)
lc = LineCollection(pn_segments, colors='k', zorder=9, transform=proj, lw=0.5, ls='-')
axs[1].add_collection(lc)
axs[1].scatter(pn_lons, pn_lats, c=ssh_cmap, zorder=10, edgecolors='k', lw=0.35, s=plt_ms, transform=proj)
axs[1].set_title('Parent Simulation', fontsize=title_fs)

# Child Simulation
cn_slps = cn_min_slps.sel(time=te_times).values
cn_lons = cn_subset.lon.sel(ncol=cn_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values
cn_lats = cn_subset.lat.sel(ncol=cn_subset.PSL.compute().argmin(dim='ncol')).sel(time=te_times).values

ssh_cmap = [sshws_color(slp, units='pa') for slp in cn_slps]
cn_points = np.array([cn_lons, cn_lats]).T.reshape(-1, 1, 2)
cn_segments = np.concatenate([cn_points[:-1], cn_points[1:]], axis=1)
lc = LineCollection(cn_segments, colors='k', zorder=9, transform=proj, lw=0.5, ls='-')
axs[2].add_collection(lc)
axs[2].scatter(cn_lons, cn_lats, c=ssh_cmap, zorder=10, edgecolors='k', lw=0.35, s=plt_ms, transform=proj)
axs[2].set_title('Child Simulation', fontsize=title_fs)

# Marker properties
mew = 0.25     # marker edge width
mec = 'k'      # marker edge color
ms = 8         # marker size
#mfc           # marker face color

td = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Tropical Depression', mfc=sshws_color(35, 'mph'), color='k', lw=0.5, ls='-')
ts = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Tropical Storm', mfc=sshws_color(50, 'mph'), color='k', lw=0.5, ls='-')
c1 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 1', mfc=sshws_color(75, 'mph'), color='k', lw=0.5, ls='-')
c2 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 2', mfc=sshws_color(100, 'mph'), color='k', lw=0.5, ls='-')
c3 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 3', mfc=sshws_color(115, 'mph'), color='k', lw=0.5, ls='-')
c4 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 4', mfc=sshws_color(135, 'mph'), color='k', lw=0.5, ls='-')
c5 = mlines.Line2D([], [], marker='o', ms=ms, mew=mew, mec=mec, label='Category 5', mfc=sshws_color(160, 'mph'), color='k', lw=0.5, ls='-')

miami_coords = (25.775163, -80.208615)
for ax in axs.ravel():
    ax.scatter(x=miami_coords[1]+360, y=miami_coords[0], zorder=6, marker='*', c='#00FF00', s=200, edgecolors='k', lw=0.5, transform=proj)
    l = ax.legend(handles = [c5, c4, c3, c2, c1, ts, td], loc='upper right', fontsize=12.5, shadow=False)
    l.set_zorder(1001)
    
#fig.suptitle('With Modified SSH MSLP for All Tracks')

plt.show()
#fig.savefig('../figs/sshs_modified_tracks.png', bbox_inches=None, dpi=300)

### Houly Accumulated Precip

In [ ]:
def precip_metrics(ds, var, bbox):
    # Slices by lat/lon bounding box
    if 'lon' in ds.dims:
        ds = ds.sel(lon=slice(int(bbox[0][0]), int(bbox[0][1])), lat=slice(int(bbox[1][0]), int(bbox[1][1])))
    else:
        try:
            ds['lon'] = ds['lon'].load()
            ds['lat'] = ds['lat'].load()
            ds = ds.where((ds['lon'] >= bbox[0][0]) & (ds['lon'] <= bbox[0][1]) &
                          (ds['lat'] >= bbox[1][0]) & (ds['lat'] <= bbox[1][1]), drop=True)
        except NotImplementedError:
            ds = ds.where((ds['lon'] >= bbox[0][0]) & (ds['lon'] <= bbox[0][1]) &
                          (ds['lat'] >= bbox[1][0]) & (ds['lat'] <= bbox[1][1]), drop=True)
        
    # Filter out > 0.01 mm rain
    
    try:
        tot_precip = ds[var].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
        max_precip = ds[var].metpy.convert_units('inch/hour').max(dim='time') * units('hour')
    except DimensionalityError:
        ds[var] = ds[var] * units('m/s')
        tot_precip = ds[var].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
        max_precip = ds[var].metpy.convert_units('inch/hour').max(dim='time') * units('hour')
    
    return tot_precip, max_precip

#### Histograms

##### Precip

In [ ]:
# Total sum (inches) and max hourly (inches)
bbox = basin_bboxes('south florida')
cn_tp, cn_mp = precip_metrics(h4cn_ds, 'PRECT', bbox)
cr_tp, cr_mp = precip_metrics(h4cr_ds, 'PRECT', bbox)
pn_tp, pn_mp = precip_metrics(h4pn_ds, 'PRECT', bbox)
pr_tp, pr_mp = precip_metrics(h4pr_ds, 'PRECT', bbox)

# TODO: Gut check max against h1xx_ds['PRECTMX'] but these are max daily precip rates (vs max hourly)

In [ ]:
def precip_plots(native_parent_sim, native_child_sim, regridded_parent_sim, regridded_child_sim, 
                 stat='frequency', suptitle=None, save=False):
    
    # Total sum (inches) and max hourly (inches)
    bbox = basin_bboxes('south florida')
    cn_tp, cn_mp = precip_metrics(native_child_sim, 'PRECT', bbox)
    cr_tp, cr_mp = precip_metrics(regridded_child_sim, 'PRECT', bbox)
    pn_tp, pn_mp = precip_metrics(native_parent_sim, 'PRECT', bbox)
    pr_tp, pr_mp = precip_metrics(regridded_parent_sim, 'PRECT', bbox)
    
    lw=1.25
    
    fig, axs = plt.subplots(2, 2, figsize=(15, 15), dpi=300, layout='constrained')
    sns.histplot(pn_tp.values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][0])
    sns.histplot(cn_tp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='MPAS (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][0])
    #axs[0][0].set_title('Native Grid Total Precipitation Distribution', fontsize=16)
    axs[0][0].set_title('Total Precipitation', fontsize=22)
    axs[0][0].tick_params(labelbottom=False)
    axs[0][0].set_ylabel('Density', fontsize=18)

    
    g3 = sns.histplot(pn_mp.values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][1])
    g4 = sns.histplot(cn_mp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='MPAS (Native)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][1])
    #axs[0][1].set_title('Native Grid Maximum Hourly Precipitation Distribution', fontsize=16)
    axs[0][1].set_title('Maximum Hourly Precipitation', fontsize=22)
    axs[0][1].tick_params(labelbottom=False)
    axs[0][1].set_ylabel(None)

    sns.histplot(pn_tp.values.flatten(), stat=stat, element='step', color='#7a8fc2', alpha=0.0005, label='CAM (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][0])
    sns.histplot(cn_tp.values.flatten(), stat=stat, element='step', color='#ff8bd1', alpha=0.0005, label='MPAS (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][0])
    #axs[1][0].set_title('Regridded Total Precipitation Distribution', fontsize=16)
    axs[1][0].set_xlabel('Accumulated Precipitation [inches]', fontsize=18)
    axs[1][0].set_ylabel('Density', fontsize=18)

    sns.histplot(pn_mp.values.flatten(), stat=stat, element='step', color='#7a8fc2', alpha=0.0005, label='CAM (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][1])
    sns.histplot(cn_mp.values.flatten(), stat=stat, element='step', color='#ff8bd1', alpha=0.0005, label='MPAS (Regridded)', 
                 kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][1])
    #axs[1][1].set_title('Regridded Maximum Hourly Precipitation Distribution', fontsize=16)
    axs[1][1].set_xlabel('Accumulated Precipitation [inches]', fontsize=18)
    axs[1][1].set_ylabel(None)
    
    for ax in axs.ravel():
        
        ax.legend(fontsize=18)
        ax.tick_params(axis='both', which='major', labelsize=16)

    if suptitle != None:
        fig.suptitle(suptitle, fontsize=16, fontweight='bold')
    plt.show()
    
    if save == True:
        fig.savefig('../figs/precip_histograms.png', bbox_inches='tight', dpi=300)

In [ ]:
#mpl.rcParams['axes.labelsize'] = 16
precip_plots(h4pn_ds, h4cn_ds, h4pr_ds, h4cr_ds, 'density', save=False)
#mpl.rcParams['axes.labelsize'] = 12

In [ ]:
h4cn_ds['PRECL'] = (h4cn_ds['PRECT'] - h4cn_ds['PRECC']) * units('m/s')
h4cr_ds['PRECL'] = (h4cr_ds['PRECT'] - h4cr_ds['PRECC']) * units('m/s')

In [ ]:
# Total sum (inches) and max hourly (inches)
bbox = basin_bboxes('south florida')
cn_convective_tp, cn_convective_mp = precip_metrics(h4cn_ds, 'PRECC', bbox)
cn_synoptic_tp, cn_synoptic_mp = precip_metrics(h4cn_ds, 'PRECL', bbox)
cn_tp, cn_mp = precip_metrics(h4cn_ds, 'PRECT', bbox)

cr_convective_tp, cr_convective_mp = precip_metrics(h4cr_ds, 'PRECC', bbox)
cr_synoptic_tp, cr_synoptic_mp = precip_metrics(h4cr_ds, 'PRECL', bbox)
cr_tp, cr_mp = precip_metrics(h4cr_ds, 'PRECT', bbox)

In [ ]:
lw=1.25
stat='density'

fig, axs = plt.subplots(2, 2, figsize=(15, 15), dpi=300, layout='constrained')
sns.histplot(cn_convective_tp.values.flatten(), stat=stat, element='step', color='#490029', alpha=0.0005, label='Convective (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][0])
sns.histplot(cn_synoptic_tp.values.flatten(), stat=stat, element='step', color='#b00075', alpha=0.0005, label='Synoptic (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][0])
sns.histplot(cn_tp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='Total (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][0])
axs[0][0].set_title('Native Grid Total Precipitation Distribution', fontsize=16)

sns.histplot(cn_convective_mp.values.flatten(), stat=stat, element='step', color='#490029', alpha=0.0005, label='Convective (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][1])
sns.histplot(cn_synoptic_mp.values.flatten(), stat=stat, element='step', color='#b00075', alpha=0.0005, label='Synoptic (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][1])
sns.histplot(cn_mp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='Total (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0][1])
axs[0][1].set_title('Native Grid Maximum Hourly Precipitation Distribution', fontsize=16)

sns.histplot(cr_convective_tp.values.flatten(), stat=stat, element='step', color='#48002b', alpha=0.0005, label='Convective (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][0])
sns.histplot(cr_synoptic_tp.values.flatten(), stat=stat, element='step', color='#8e3767', alpha=0.0005, label='Synoptic (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][0])
sns.histplot(cr_tp.values.flatten(), stat=stat, element='step', color='#ff8bd1', alpha=0.0005, label='Total (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][0])
axs[1][0].set_title('Regridded Total Precipitation Distribution', fontsize=16)

sns.histplot(cr_convective_mp.values.flatten(), stat=stat, element='step', color='#48002b', alpha=0.0005, label='Convective (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][1])
sns.histplot(cr_synoptic_mp.values.flatten(), stat=stat, element='step', color='#8e3767', alpha=0.0005, label='Synoptic (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][1])
sns.histplot(cr_mp.values.flatten(), stat=stat, element='step', color='#ff8bd1', alpha=0.0005, label='Total (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1][1])
axs[1][1].set_title('Regridded Maximum Hourly Precipitation Distribution', fontsize=16)

for ax in axs.ravel():
    ax.set_xlabel('Accumulated Precipitation [inches]')
    ax.legend(fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=12)

fig.suptitle('Convective vs. Synoptic Precip (MPAS)', fontsize=16, fontweight='bold')
plt.show()

##### Depth Duration Frequency

In [ ]:
# 4 curves, I think? Native/Regridded Parent/Child
# Rainfall depth (accumulated precip) on the y-axis
# Time on x-axis (log format)

fig, ax = plt.subplots(figsize=(12, 7), dpi=200)
times = h3cr_ds.time.dt.strftime("%Y-%m-%d %H:%M").values

ax.plot(times, pn_tp.values, label='CAM (Native)', c='#024588')
ax.plot(times, pr_tp.values, label='CAM (Regridded)', c='#7a8fc2', ls='--', lw=0.75)
ax.plot(times, cn_tp.values, label='MPAS (Native)', c='#FA03B2')
ax.plot(times, cr_tp.values, label='MPAS (Regridded)', c='#ff8bd1', ls='--', lw=0.75)

ax.set_ylabel('Accumulated Precipitation [in]', fontsize=14)
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(start, end, 2))
plt.setp(ax.get_xticklabels(), ha='right', rotation=45)
ax.set_title('Accumulated Precipitation', fontsize=16)
ax.legend(fontsize=12)
plt.show()

In [ ]:
pr_tp.plot.line(x='time')

##### Wind

In [ ]:
def wind_metrics(ds, var, bbox):
    if 'lon' in ds.dims:
        ds = ds.sel(lon=slice(int(bbox[0][0]), int(bbox[0][1])), lat=slice(int(bbox[1][0]), int(bbox[1][1])))
    else:
        try:
            ds = ds.where((ds['lon'] >= bbox[0][0]) & (ds['lon'] <= bbox[0][1]) &
                          (ds['lat'] >= bbox[1][0]) & (ds['lat'] <= bbox[1][1]), drop=True)
        except NotImplementedError:
            ds['lon'] = ds['lon'].load()
            ds['lat'] = ds['lat'].load()
            ds = ds.where((ds['lon'] >= bbox[0][0]) & (ds['lon'] <= bbox[0][1]) &
                          (ds['lat'] >= bbox[1][0]) & (ds['lat'] <= bbox[1][1]), drop=True)
    
    try:
        max_wsp = ds[var].metpy.convert_units('mph').max(dim='time') * units('mph')
    except DimensionalityError:
        ds[var] = ds[var] * units('m/s')
        max_wsp = ds[var].metpy.convert_units('mph').max(dim='time') * units('mph')
        
    return max_wsp

In [ ]:
bbox = basin_bboxes('south florida')
cn_wsp = wind_metrics(h3cn_ds, 'U10', bbox)
cr_wsp = wind_metrics(h3cr_ds, 'U10', bbox)
pn_wsp = wind_metrics(h3pn_ds, 'U10', bbox)
pr_wsp = wind_metrics(h3pr_ds, 'U10', bbox)

lw=1.25
stat='density'

fig, axs = plt.subplots(2, 1, figsize=(10, 10), dpi=300, layout='constrained')
sns.histplot(pn_wsp.values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0])
sns.histplot(cn_wsp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='MPAS (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0])
axs[0].set_title('Native Grid Maximum Wind Speed Distribution - Surface', fontsize=16)

sns.histplot(pr_wsp.values.flatten(), stat=stat, element='step', color='#7a8fc2', alpha=0.0005, label='CAM (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1])
sns.histplot(cr_wsp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='MPAS (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1])
axs[1].set_title('Regridded Maximum Wind Speed Distribution - Surface', fontsize=16)

for ax in axs.ravel():
    ax.set_xlabel('Maximum Wind Speed [mph]')
    ax.legend(fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=12)

# if suptitle != None:
#     fig.suptitle(suptitle, fontsize=16, fontweight='bold')
plt.show()

In [ ]:


var = 'WSP850'
cn_wsp = wind_metrics(h3cn_ds, var, bbox_bahamas)
cr_wsp = wind_metrics(h3cr_ds, var, bbox_bahamas)
pn_wsp = wind_metrics(h3pn_ds, var, bbox_bahamas)
pr_wsp = wind_metrics(h3pr_ds, var, bbox_bahamas)

lw=1.25
stat='density'

fig, axs = plt.subplots(2, 1, figsize=(10, 10), dpi=300, layout='constrained')
sns.histplot(pn_wsp.values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0])
sns.histplot(cn_wsp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='MPAS (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0])
axs[0].set_title(f'Native Grid Maximum Wind Speed Distribution - {var}', fontsize=16)

sns.histplot(pr_wsp.values.flatten(), stat=stat, element='step', color='#7a8fc2', alpha=0.0005, label='CAM (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1])
sns.histplot(cr_wsp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='MPAS (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1])
axs[1].set_title(f'Regridded Maximum Wind Speed Distribution - {var}', fontsize=16)

for ax in axs.ravel():
    ax.set_xlabel('Maximum Wind Speed [mph]')
    ax.legend(fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=12)

# if suptitle != None:
#     fig.suptitle(suptitle, fontsize=16, fontweight='bold')
plt.show()

In [ ]:
bbox = basin_bboxes('south florida')
cn_wsp = wind_metrics(h3cn_ds, 'WSP850', bbox)
cr_wsp = wind_metrics(h3cr_ds, 'WSP850', bbox)
pn_wsp = wind_metrics(h3pn_ds, 'WSP850', bbox)
pr_wsp = wind_metrics(h3pr_ds, 'WSP850', bbox)

lw=1.25
stat='density'

fig, axs = plt.subplots(2, 1, figsize=(10, 10), dpi=300, layout='constrained')
sns.histplot(pn_wsp.values.flatten(), stat=stat, element='step', color='#024588', alpha=0.0005, label='CAM (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0])
sns.histplot(cn_wsp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='MPAS (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[0])
axs[0].set_title('Native Grid Maximum Wind Speed Distribution - 850 mb', fontsize=16)

sns.histplot(pr_wsp.values.flatten(), stat=stat, element='step', color='#7a8fc2', alpha=0.0005, label='CAM (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1])
sns.histplot(cr_wsp.values.flatten(), stat=stat, element='step', color='#FA03B2', alpha=0.0005, label='MPAS (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':lw}, lw=lw, ax=axs[1])
axs[1].set_title('Regridded Maximum Wind Speed Distribution - 850 mb', fontsize=16)

for ax in axs.ravel():
    ax.set_xlabel('Maximum Wind Speed [mph]')
    ax.legend(fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=12)

# if suptitle != None:
#     fig.suptitle(suptitle, fontsize=16, fontweight='bold')
plt.show()

In [ ]:
def plot_windswaths(parent_data, child_data, var, bbox):
    proj=ccrs.PlateCarree()
    fig, axs = plt.subplots(1, 2, figsize=(12.5,14), dpi=300, subplot_kw=dict(projection=proj), layout='constrained')

    alpha = 0.75
    norm=None

    cr_wsp = wind_metrics(child_data, var, bbox)
    pr_wsp = wind_metrics(parent_data, var, bbox)

    lons, lats = basin_bboxes('florida')

    vmin = cr_wsp.values.min()
    vmax = cr_wsp.values.max()
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

    pr_wsp.plot(ax=axs[0], add_colorbar=False, alpha=alpha, norm=norm, transform=proj)
    p1 = axs[0].set_title(f"CAM (Regridded) Maximum {var} WSP Distribution")

    cr_wsp.plot(ax=axs[1], add_colorbar=False, alpha=alpha, norm=norm, transform=proj)
    axs[1].set_title(f"MPAS (Regridded) Maximum {var} WSP Distribution")

    for ax in axs.ravel():
        ax.set_extent([lons[0], lons[1], lats[0], lats[1]])
        ax.coastlines(resolution='10m', linewidth=1, edgecolor='#323232')
        ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=1, facecolor='none', edgecolor='#323232', zorder=2)

    fig.colorbar(cm.ScalarMappable(norm=norm, cmap='viridis'), orientation='horizontal',
                 ax=axs, label='Maximum Wind Speed [mph]', shrink=0.75)

    plt.show()

In [ ]:
basin_bboxes('florida')

In [ ]:
np.mod(287.5 - 180.0, 360.0) - 180.0

In [ ]:
#WESN
#lon_e = -75.25
lon_e = -75.0
lon_e = np.mod(lon_e + 180.0, 360.0) + 180.0
#lon_w = -77.0
lon_w = 
lon_w = np.mod(lon_w + 180.0, 360.0) + 180.0
lat_n = 26.25
lat_s = 23.75

lons = tuple((lon_w, lon_e))
lats = tuple((lat_s, lat_n))

bbox_bahamas = tuple((lons, lats))

plot_windswaths(h3pr_ds, h3cr_ds, 'WSP850', bbox_bahamas)

In [ ]:
bbox = basin_bboxes('south florida')
plot_windswaths(h3pr_ds, h3cr_ds, 'U10', bbox)

In [ ]:
bbox = basin_bboxes('florida')
plot_windswaths(h3pr_ds, h3cr_ds, 'U10', bbox)

In [ ]:
bbox = basin_bboxes('south florida')
plot_windswaths(h3pr_ds, h3cr_ds, 'WSP850', bbox)

In [ ]:
bbox = basin_bboxes('florida')
plot_windswaths(h3pr_ds, h3cr_ds, 'WSP850', bbox)

#### Trials

In [ ]:
precip_plots(h4pn_ds, h4cn_ds, h4pr_ds, h4cr_ds, 'percent')

In [ ]:
# Plots frequency density (aka the area under the curve = 1, bin width * density = % of data/100) with unequal number of bins.
precip_plots(h4pn_ds, h4cn_ds, h4pr_ds, h4cr_ds, 'density', 'Frequency Density (Equal Bin Ratio, Unequal Number of Bins)')

In [ ]:
# Frequency with same number of bins, but unequal bin sizes
bbox = basin_bboxes('south florida')
cn_tp, cn_mp = precip_metrics(h4cn_ds, 'PRECT', bbox)
cr_tp, cr_mp = precip_metrics(h4cr_ds, 'PRECT', bbox)
pn_tp, pn_mp = precip_metrics(h4pn_ds, 'PRECT', bbox)
pr_tp, pr_mp = precip_metrics(h4pr_ds, 'PRECT', bbox)

stat='frequency'
bins=10
fig, axs = plt.subplots(2, 2, figsize=(15, 15), dpi=200, layout='constrained')
sns.histplot(pn_tp.values.flatten(), stat=stat, bins=bins, element='step', color='#003f5c', alpha=0.0005, label='CAM (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][0])
sns.histplot(cn_tp.values.flatten(), stat=stat, bins=bins, element='step', color='#ff6361', alpha=0.0005, label='MPAS (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][0])
axs[0][0].set_title('Native Grid Total Precipitation Distribution')

sns.histplot(pn_mp.values.flatten(), stat=stat, bins=bins, element='step', color='#003f5c', alpha=0.0005, label='CAM (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][1])
sns.histplot(cn_mp.values.flatten(), stat=stat, bins=bins, element='step', color='#ff6361', alpha=0.0005, label='MPAS (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][1])
axs[0][1].set_title('Native Grid Maximum Hourly Precipitation Distribution')

sns.histplot(pn_tp.values.flatten(), stat=stat, bins=bins, element='step', color='#003f5c', alpha=0.0005, label='CAM (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][0])
sns.histplot(cn_tp.values.flatten(), stat=stat, bins=bins, element='step', color='#ff6361', alpha=0.0005, label='MPAS (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][0])
axs[1][0].set_title('Regridded Total Precipitation Distribution')

sns.histplot(pn_mp.values.flatten(), stat=stat, bins=bins, element='step', color='#003f5c', alpha=0.0005, label='CAM (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][1])
sns.histplot(cn_mp.values.flatten(), stat=stat, bins=bins, element='step', color='#ff6361', alpha=0.0005, label='MPAS (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][1])
axs[1][1].set_title('Regridded Maximum Hourly Precipitation Distribution')

for ax in axs.ravel():
    ax.set_xlabel('Accumulated Precipitation [inches]')
    ax.legend()

fig.suptitle('Frequency with Equal Number of Bins (10), Unequal Bin Ratio', fontsize=16, fontweight='bold')
plt.show()

In [ ]:
# Frequency density with same number of bins
bbox = basin_bboxes('south florida')
cn_tp, cn_mp = precip_metrics(h4cn_ds, 'PRECT', bbox)
cr_tp, cr_mp = precip_metrics(h4cr_ds, 'PRECT', bbox)
pn_tp, pn_mp = precip_metrics(h4pn_ds, 'PRECT', bbox)
pr_tp, pr_mp = precip_metrics(h4pr_ds, 'PRECT', bbox)

stat='density'
bins=10
fig, axs = plt.subplots(2, 2, figsize=(15, 15), dpi=200, layout='constrained')
sns.histplot(pn_tp.values.flatten(), stat=stat, bins=bins, element='step', color='#003f5c', alpha=0.0005, label='CAM (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][0])
sns.histplot(cn_tp.values.flatten(), stat=stat, bins=bins, element='step', color='#ff6361', alpha=0.0005, label='MPAS (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][0])
axs[0][0].set_title('Native Grid Total Precipitation Distribution')

sns.histplot(pn_mp.values.flatten(), stat=stat, bins=bins, element='step', color='#003f5c', alpha=0.0005, label='CAM (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][1])
sns.histplot(cn_mp.values.flatten(), stat=stat, bins=bins, element='step', color='#ff6361', alpha=0.0005, label='MPAS (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][1])
axs[0][1].set_title('Native Grid Maximum Hourly Precipitation Distribution')

sns.histplot(pn_tp.values.flatten(), stat=stat, bins=bins, element='step', color='#003f5c', alpha=0.0005, label='CAM (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][0])
sns.histplot(cn_tp.values.flatten(), stat=stat, bins=bins, element='step', color='#ff6361', alpha=0.0005, label='MPAS (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][0])
axs[1][0].set_title('Regridded Total Precipitation Distribution')

sns.histplot(pn_mp.values.flatten(), stat=stat, bins=bins, element='step', color='#003f5c', alpha=0.0005, label='CAM (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][1])
sns.histplot(cn_mp.values.flatten(), stat=stat, bins=bins, element='step', color='#ff6361', alpha=0.0005, label='MPAS (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][1])
axs[1][1].set_title('Regridded Maximum Hourly Precipitation Distribution')

for ax in axs.ravel():
    ax.set_xlabel('Accumulated Precipitation [inches]')
    ax.legend()

fig.suptitle('Frequency Density with Equal Number of Bins (10) and Equal Bin Ratio', fontsize=16, fontweight='bold')
plt.show()

In [ ]:
stat='percent'
fig, axs = plt.subplots(2, 2, figsize=(15, 15), dpi=200, layout='constrained')
sns.histplot(pn_tp.values.flatten(), stat=stat, element='step', color='#003f5c', alpha=0.0005, label='CAM (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][0])
sns.histplot(pr_tp.values.flatten(), stat=stat, bins=bins, element='step', color='#ff6361', alpha=0.0005, label='CAM (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][0])
axs[0][0].set_title('CAM Total Precipitation Distribution')

sns.histplot(pn_mp.values.flatten(), stat=stat, element='step', color='#003f5c', alpha=0.0005, label='CAM (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][1])
sns.histplot(pr_mp.values.flatten(), stat=stat, element='step', color='#ff6361', alpha=0.0005, label='CAM (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[0][1])
axs[0][1].set_title('CAM Maximum Hourly Precipitation Distribution')

sns.histplot(cn_tp.values.flatten(), stat=stat, element='step', color='#003f5c', alpha=0.0005, label='MPAS (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][0])
sns.histplot(cr_tp.values.flatten(), stat=stat, element='step', color='#ff6361', alpha=0.0005, label='MPAS (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][0])
axs[1][0].set_title('MPAS Total Precipitation Distribution')

sns.histplot(cn_mp.values.flatten(), stat=stat, element='step', color='#003f5c', alpha=0.0005, label='MPAS (Native)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][1])
sns.histplot(cr_mp.values.flatten(), stat=stat, element='step', color='#ff6361', alpha=0.0005, label='MPAS (Regridded)', 
             kde=True, line_kws={'ls':'--', 'lw':1}, lw=1, ax=axs[1][1])
axs[1][1].set_title('MPAS Maximum Hourly Precipitation Distribution')

for ax in axs.ravel():
    ax.set_xlabel('Accumulated Precipitation [inches]')
    ax.legend()

fig.suptitle('Frequency Density with Equal Number of Bins (10) and Equal Bin Ratio', fontsize=16, fontweight='bold')
plt.show()

#### Maps

##### Matplotlib

In [ ]:
# Based off of https://water.weather.gov/precip/ 30-day observed precip scale
nws_precip_colors = [
    '#ffffff',  # 0 - 0.01 inches
    "#4bd2f7",  # 0.01 - 0.10 inches  light blue
    "#699fd0",  # 0.10 - 0.25 inches  mid blue
    "#3c4bac",  # 0.25 - 0.50 inches  dark blue
    "#3cf74b",  # 0.50 - 1.00 inches  light green
    "#3cb447",  # 1.00 - 1.50 inches  mid green
    "#3c8743",  # 1.50 - 2.00 inches  dark green
    "#1f4723",  # 2.00 - 3.00 inches  darkest green
    "#f7f73c",  # 3.00 - 4.00 inches  yellow
    "#fbde88",  # 4.00 - 5.00 inches  weird tan
    "#f7ac3c",  # 5.00 - 6.00 inches  orange
    "#c47908",  # 6.00 - 8.00 inches  dark orange
    "#f73c3c",  # 8.00 - 10.00 inch   red
    "#bf3c3c",  # 10.00 - 15.00 inch  mid red
    "#6e2b2b",  # 15.00 - 20.00 inch  dark red 
    "#f73cf7",  # 20.00 - 25.00 inch  bright pink
    "#9974e4",  # 25.00 - 30.00 inch  purple
    "#e1e1e1",  # 30.00 - 40.00 inch  gray
    ]
nws_cmap = mpl.colors.ListedColormap(nws_precip_colors, 'nws_precip')
levels = [0.0, 0.01, 0.10, 0.25, 0.50, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0]
norm = mpl.colors.BoundaryNorm(boundaries=levels, ncolors=len(levels))
mpl.colormaps.register(nws_cmap, force=True)
nws_cmap

In [ ]:
# Total sum (inches) and max hourly (inches)
bbox = basin_bboxes('florida')
cn_tp, cn_mp = precip_metrics(h4cn_ds, 'PRECT', bbox)
cr_tp, cr_mp = precip_metrics(h4cr_ds, 'PRECT', bbox)
pn_tp, pn_mp = precip_metrics(h4pn_ds, 'PRECT', bbox)
pr_tp, pr_mp = precip_metrics(h4pr_ds, 'PRECT', bbox)

In [ ]:
proj=ccrs.PlateCarree()

# bbox = basin_bboxes('florida')
# cn_tp, cn_mp = precip_metrics(h4cn_ds, 'PRECT', bbox)
# cr_tp, cr_mp = precip_metrics(h4cr_ds, 'PRECT', bbox)
# pn_tp, pn_mp = precip_metrics(h4pn_ds, 'PRECT', bbox)
# pr_tp, pr_mp = precip_metrics(h4pr_ds, 'PRECT', bbox)

lons = tuple((cr_tp.lon.values.min(), cr_tp.lon.values.max()))
lats = tuple((cr_tp.lat.values.min(), cr_tp.lat.values.max()))

fig, axs = plt.subplots(2, 2, figsize=(14,12), dpi=300, subplot_kw=dict(projection=proj), layout='constrained')

ts = 18
alpha = 0.75
pr_tp.plot(ax=axs[0][0], norm=norm, cmap=nws_cmap, add_colorbar=False, alpha=alpha)
axs[0][0].set_title('CAM (Regridded) Total Precipitation', fontsize=ts)

pr_mp.plot(ax=axs[0][1], norm=norm, cmap=nws_cmap, add_colorbar=False, alpha=alpha)
axs[0][1].set_title('CAM (Regridded) Maximum Hourly Precipitation', fontsize=ts)

cr_tp.plot(ax=axs[1][0], norm=norm, cmap=nws_cmap, add_colorbar=False, alpha=alpha)
axs[1][0].set_title('MPAS (Regridded) Total Precipitation', fontsize=ts)

cr_mp.plot(ax=axs[1][1], norm=norm, cmap=nws_cmap, add_colorbar=False, alpha=alpha)
axs[1][1].set_title('MPAS (Regridded) Maximum Hourly Precipitation', fontsize=ts)

for ax in axs.ravel():
    ax.set_extent([lons[0], lons[1], lats[0], lats[1]])
    ax.coastlines(resolution='10m', linewidth=1, edgecolor='#323232')
    ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=1, facecolor='none', edgecolor='#323232', zorder=2)
    
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=nws_cmap), orientation='horizontal', 
             ax=axs, ticks=levels)
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label='Accumulated Precipitation [inches]', size=16, fontstyle='italic')


plt.show()
#fig.savefig('../figs/precip_dist_map_full.png', bbox_inches=None, dpi=300, transparent=False)

In [ ]:
proj=ccrs.PlateCarree()

# lons, lats = basin_bboxes('florida')
bbox = basin_bboxes('south florida')

cn_tp, cn_mp = precip_metrics(h4cn_ds, 'PRECT', bbox)
cr_tp, cr_mp = precip_metrics(h4cr_ds, 'PRECT', bbox)
pn_tp, pn_mp = precip_metrics(h4pn_ds, 'PRECT', bbox)
pr_tp, pr_mp = precip_metrics(h4pr_ds, 'PRECT', bbox)

fig, axs = plt.subplots(2, 2, figsize=(14,12.5), dpi=200, subplot_kw=dict(projection=proj), layout='constrained')

pr_tp.plot(ax=axs[0][0], norm=norm, cmap=nws_cmap, add_colorbar=False)
axs[0][0].set_title('CAM (Regridded) Total Accumulated Precipitation')

pr_mp.plot(ax=axs[0][1], norm=norm, cmap=nws_cmap, add_colorbar=False)
axs[0][1].set_title('CAM (Regridded) Maximum Hourly Accumulated Precipitation')

cr_tp.plot(ax=axs[1][0], norm=norm, cmap=nws_cmap, add_colorbar=False)
axs[1][0].set_title('MPAS (Regridded) Total Accumulated Precipitation')

cr_mp.plot(ax=axs[1][1], norm=norm, cmap=nws_cmap, add_colorbar=False)
axs[1][1].set_title('MPAS (Regridded) Maximum Hourly Accumulated Precipitation')

for ax in axs.ravel():
    ax.set_extent([lons[0], lons[1], lats[0], lats[1]])
    ax.coastlines(resolution='10m', linewidth=0.5, edgecolor='#323232')
    ax.add_feature(cfeature.LAKES.with_scale('10m'), linewidth=0.5, facecolor='none', edgecolor='#323232', zorder=2)
    
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=nws_cmap), orientation='horizontal', ax=axs, ticks=levels,
             label='Accumulated Precipitation [inches]', shrink=0.75)
#mpl.cm.ScalarMappable(norm=norm, cmap=nws_cmap)
plt.show()
#fig.savefig('../figs/precip_dist_map_clipped.png', bbox_inches=None, dpi=300, transparent=False)

##### Datashader

In [ ]:
h4cn_ds['tot_prect'] = h4cn_ds['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
h4cn_ds['max_hrly_prect'] = h4cn_ds['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')

h4pn_ds['tot_prect'] = h4pn_ds['PRECT'].metpy.convert_units('inch/hour').sum(dim='time') * units('hour')
h4pn_ds['max_hrly_prect'] = h4pn_ds['PRECT'].metpy.convert_units('inch/hour').max(dim='time') * units('hour')

In [ ]:
def unzipMesh(x,tris,t):
    return tris[(np.abs((x[tris[:,0]])-(x[tris[:,1]])) < t) & (np.abs((x[tris[:,0]])-(x[tris[:,2]])) < t)]

def triArea(x,y,tris):
    return ((x[tris[:,1]]-x[tris[:,0]]) * (y[tris[:,2]]-y[tris[:,0]])) - ((x[tris[:,2]]-x[tris[:,0]]) * (y[tris[:,1]]-y[tris[:,0]]))

def orderCCW(x,y,tris):
    tris[triArea(x,y,tris)<0.0,:] = tris[triArea(x,y,tris)<0.0,::-1]
    return(tris)

def createHVTriMesh(x,y,triangle_indices, var, var_name, n_workers=1):
    verts = np.column_stack([x, y, var])
    verts_df  = pd.DataFrame(verts,  columns=['Longitude', 'Latitude', var_name])
    tris_df   = pd.DataFrame(triangle_indices, columns=['v0', 'v1', 'v2'])
    verts_ddf = dd.from_pandas(verts_df, npartitions=n_workers)
    tris_ddf = dd.from_pandas(tris_df, npartitions=n_workers)
    tri_nodes = hv.Nodes(verts_ddf, ['Longitude', 'Latitude', 'index'], [var_name])
    trimesh = hv.TriMesh((tris_ddf, tri_nodes))
    return(trimesh)

lonCell = np.mod(np.rad2deg(c_mesh_ds['lonCell'].values) - 180.0, 360.0) - 180.0
latCell = np.rad2deg(c_mesh_ds['latCell'].values)

lonVertex = np.mod(np.rad2deg(c_mesh_ds['lonVertex'].values) - 180.0, 360.0) - 180.0
latVertex = np.rad2deg(c_mesh_ds['latVertex'].values)

tris = c_mesh_ds.cellsOnVertex.values - 1

tris_ccw = orderCCW(lonCell,latCell,tris)

tris_ccw_flat = unzipMesh(lonCell,tris,90.0)

#sel_time = 23
tvar = 'tot_prect'
primalVar = h4cn_ds[tvar].where(h4cn_ds[tvar].values >= 0.01)
primalVar = primalVar.values

lon_range, lat_range = basin_bboxes('florida')

project_coords = True
if project_coords == True:
    proj = ccrs.Robinson(central_longitude=0.0)
    xPCS, yPCS, _ = proj.transform_points(ccrs.PlateCarree(), lonCell, latCell).T
    trimesh = createHVTriMesh(xPCS,yPCS,tris_ccw_flat, primalVar, tvar, n_workers=1)
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    lon_range = tuple(x_range)
    lat_range = tuple(y_range)

else:
    proj = ccrs.PlateCarree()
    lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
    trimesh = createHVTriMesh(lonCell,latCell,tris_ccw, primalVar, 'FLUT', n_workers=1)

rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)

In [ ]:
mpas_total = rasterized.opts(tools=['hover'], colorbar=True, cmap=nws_cmap, title='MPAS (Native) Total Precipitation', 
                                    clim=(levels[0], levels[-1]), width=1000, height=800, color_levels=levels, 
                                    clipping_colors={'NaN':'#FFFFFF'}, clabel='Accumulated Precipitation [inches]', colorbar_position='bottom', 
                                    fontsize=dict(title='30pt', clabel='28pt', cticks='26pt'), alpha=0.75) *\
gf.coastline(projection=proj, line_width=2.0, alpha=1).options(scale='10m') *\
gf.lakes(projection=proj, line_width=2.0, fill_color='none', line_color='#323232', alpha=1).options(scale='10m')


In [ ]:
primalVar = h4cn_ds['max_hrly_prect'].where(h4cn_ds['max_hrly_prect'].values >= 0.01)
primalVar = primalVar.values

tvar = 'max_hrly_prect'
lon_range, lat_range = basin_bboxes('florida')

project_coords = True
if project_coords == True:
    proj = ccrs.Robinson(central_longitude=0.0)
    xPCS, yPCS, _ = proj.transform_points(ccrs.PlateCarree(), lonCell, latCell).T
    trimesh = createHVTriMesh(xPCS,yPCS,tris_ccw_flat, primalVar, tvar, n_workers=1)
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    lon_range = tuple(x_range)
    lat_range = tuple(y_range)

else:
    proj = ccrs.PlateCarree()
    lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
    trimesh = createHVTriMesh(lonCell,latCell,tris_ccw, primalVar, tvar, n_workers=1)

rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)

mpas_max_hrly = rasterized.opts(tools=['hover'], colorbar=False, cmap=nws_cmap, title='MPAS (Native) Maximum Hourly Precipitation', 
                                clim=(levels[0], levels[-1]), width=1000, height=800, color_levels=levels, clipping_colors={'NaN':'#FFFFFF'},
                                fontsize=dict(title='30pt'), alpha=0.75) *\
gf.coastline(projection=proj, line_width=2.0, alpha=1).options(scale='10m') *\
gf.lakes(projection=proj, line_width=2.0, fill_color='none', line_color='#323232', alpha=1).options(scale='10m')

In [ ]:
def triangulate(vertices, x="Longitude", y="Latitude"):
    """
    Generate a triangular mesh for the given x,y,z vertices, using Delaunay triangulation.
    For large n, typically results in about double the number of triangles as vertices.
    """
    triang = Delaunay(vertices[[x,y]].values)
    print('Given', len(vertices), "vertices, created", len(triang.simplices), 'triangles.')
    
    tris_df = pd.DataFrame(triang.simplices, columns=['v0', 'v1', 'v2'])
    
    return tris_df

def createHVTriMesh(x, y, var, var_name, n_workers=1):
    # Declare verts array
    # This is essentally an XYZ matrix (at the location of x and y, z=?) 3 X len(verts) array
    verts = np.column_stack([x, y, var])

    # Convert to pandas
    verts_df  = pd.DataFrame(verts,  columns=['Longitude', 'Latitude', var_name])
    
    # Creates Delaunay triangular mesh
    tris_df = triangulate(verts_df)

    # Convert to dask
    verts_ddf = dd.from_pandas(verts_df, npartitions=n_workers)
    tris_ddf = dd.from_pandas(tris_df, npartitions=n_workers)

    # Declare HoloViews element
    tri_nodes = hv.Nodes(verts_ddf, ['Longitude', 'Latitude', 'index'], [var_name])
    trimesh = hv.TriMesh((tris_ddf, tri_nodes))
    return(trimesh)

# Grid centerpoints
lonCell = p_mesh_ds.grid_center_lon.values
latCell = p_mesh_ds.grid_center_lat.values
lonCell = ((lonCell - 180.0) % 360.0) - 180.0

primalVar = h4pn_ds['max_hrly_prect'].where(h4pn_ds['max_hrly_prect'].values >= 0.01)
primalVar = primalVar.values

tvar = 'max_hrly_prect'
lon_range, lat_range = basin_bboxes('florida')

project_coords = True

if project_coords == True:
    proj = ccrs.Robinson(central_longitude=0.0)
    # Creates Delaunay triangular mesh
    xPCS, yPCS, _ = proj.transform_points(ccrs.PlateCarree(), lonCell, latCell).T
    trimesh = createHVTriMesh(xPCS, yPCS, primalVar, tvar)
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    lon_range = tuple(x_range)
    lat_range = tuple(y_range)
    
else:
    proj = ccrs.PlateCarree()
    lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
    # Creates Delaunay triangular mesh
    trimesh = createHVTriMesh(lonCell, latCell, primalVar, tvar)


rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, x_range=lon_range, y_range=lat_range, vdim_prefix='')

cam_max_hrly = rasterized.opts(tools=['hover'], colorbar=False, cmap=nws_cmap, title='CAM5 (Native) Maximum Hourly Precipitation', 
                               clim=(levels[0], levels[-1]), width=1000, height=700, color_levels=levels, clipping_colors={'NaN':'#FFFFFF'}, 
                               fontsize=dict(title='30pt'), alpha=0.75) *\
gf.coastline(projection=proj, line_width=2.0, alpha=1).options(scale='10m') *\
gf.lakes(projection=proj, line_width=2.0, fill_color='none', line_color='#323232', alpha=1).options(scale='10m')

In [ ]:
primalVar = h4pn_ds['tot_prect'].where(h4pn_ds['tot_prect'].values >= 0.01)
primalVar = primalVar.values

tvar = 'tot_prect'
lon_range, lat_range = basin_bboxes('florida')

project_coords = True

if project_coords == True:
    proj = ccrs.Robinson(central_longitude=0.0)
    # Creates Delaunay triangular mesh
    xPCS, yPCS, _ = proj.transform_points(ccrs.PlateCarree(), lonCell, latCell).T
    trimesh = createHVTriMesh(xPCS, yPCS, primalVar, tvar)
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    lon_range = tuple(x_range)
    lat_range = tuple(y_range)
    
else:
    proj = ccrs.PlateCarree()
    lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
    # Creates Delaunay triangular mesh
    trimesh = createHVTriMesh(lonCell, latCell, primalVar, tvar)

# rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, x_range=lon_range, y_range=lat_range, vdim_prefix='')

# final = rasterized.opts(tools=['hover'], colorbar=True, cmap=nws_cmap, title='CAM5 (Native) Total Accumulated Precipitation', 
#                         clim=(levels[0], levels[-1]), width=1000, clabel='Accumulated Precipitation [inches]', color_levels=levels,
#                         clipping_colors={'NaN':'#FFFFFF'}, fontsize=dict(title='16pt', cticks='10pt'), alpha=0.75) *\
# gf.coastline(projection=proj, line_width=1, alpha=1).options(scale='10m') *\
# gf.lakes(projection=proj, line_width=0.5, fill_color='none', line_color='#323232', alpha=1).options(scale='10m')

# final

rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, x_range=lon_range, y_range=lat_range, vdim_prefix='')

cam_total = rasterized.opts(tools=['hover'], colorbar=False, cmap=nws_cmap, title='CAM5 (Native) Total Precipitation', 
                        clim=(levels[0], levels[-1]), width=1000, height=700, color_levels=levels, clipping_colors={'NaN':'#FFFFFF'}, 
                            fontsize=dict(title='30pt'), alpha=0.75) *\
gf.coastline(projection=proj, line_width=2.0, alpha=1).options(scale='10m') *\
gf.lakes(projection=proj, line_width=2.0, fill_color='none', line_color='#323232', alpha=1).options(scale='10m')

In [ ]:
final_plot = hv.Layout([cam_total, cam_max_hrly, mpas_total, mpas_max_hrly]).cols(2)

In [ ]:
hv.save(final_plot, '../figs/native_precip.png', fmt='png')

In [ ]:
cam_total + cam_max_hrly

In [ ]:
mpas_native_total + mpas_max_hrly

## OLR

In [ ]:
import holoviews as hv
from holoviews import opts

import geoviews as gv
import geoviews.feature as gf

import datashader as ds
from holoviews.operation.datashader import rasterize as hds_rasterize
from holoviews.operation.datashader import datashade as hds_datashade

gv.extension("bokeh","matplotlib")
#hv.extension("bokeh","matplotlib")

opts.defaults(
    opts.Image(width=1200, height=600),
    opts.RGB(width=1200, height=600))

In [ ]:
# Datashader allows for custom colormap, but not custom registered colormaps. Holoviews is the opposite.

# https://cimss.ssec.wisc.edu/satellite-blog/wp-content/uploads/sites/5/2017/09/GOES16_CleanWindow_Landfall-20170920_0957_1136anim.gif
def T_to_FLUT(T, unit='K'):
    if unit == 'C':
        T += 273.15
    sigma = 5.6693E-8
    olr = sigma*(T**4)
    
    return olr

# Normalized cimss scale
bw_colors = [(0, '#BCBCBC'), (1, '#000000')]  
bw_cmp = LinearSegmentedColormap.from_list('FLUT bw', bw_colors, N=435)

levels = np.array([T_to_FLUT(temp, 'C') for temp in [-110, -105, -87.5, -80, -70, -60, -50, -35, -27.5, -22.5]])
fracs = levels-T_to_FLUT(-110, 'C')

rainbow_colors = [(0, '#0febff'), # cyan
                  ((fracs[1]/fracs[-1]), '#7f007f'), # purple
                  ((fracs[2]/fracs[-1]), '#e5e4e5'), # white
                  ((fracs[3]/fracs[-1]), '#000000'), # black
                  ((fracs[4]/fracs[-1]), '#ff0000'), # red
                  ((fracs[5]/fracs[-1]), '#FFFF00'), # yellow
                  ((fracs[6]/fracs[-1]), '#00FF00'), # green
                  ((fracs[7]/fracs[-1]), '#000073'), # navy
                  (1, '#00ffff')] # cyan


rainbow_cmp = LinearSegmentedColormap.from_list('FLUT colors', rainbow_colors, N=184)

bws = plt.get_cmap(bw_cmp)
bws_colors = bws(np.linspace(0, 1, 435))
rainbow = plt.get_cmap(rainbow_cmp)
r_colors = rainbow(np.linspace(0, 1, 184))

all_colors = np.vstack((r_colors, bws_colors))
flut_cimss = LinearSegmentedColormap.from_list('FLUT CIMSS', all_colors)

mpl.colormaps.register(flut_cimss, name='FLUT_CIMSS', force=True)
flut_cimss

## MPAS

In [ ]:
### This funtion splits a global mesh along longitude
#
# Examine the X coordinates of each triangle in 'tris'. Return an array of 'tris' where only those triangles
# with legs whose length is less than 't' are returned.
#
def unzipMesh(x,tris,t):
    return tris[(np.abs((x[tris[:,0]])-(x[tris[:,1]])) < t) & (np.abs((x[tris[:,0]])-(x[tris[:,2]])) < t)]

# Compute the signed area of a triangle
#
def triArea(x,y,tris):
    return ((x[tris[:,1]]-x[tris[:,0]]) * (y[tris[:,2]]-y[tris[:,0]])) - ((x[tris[:,2]]-x[tris[:,0]]) * (y[tris[:,1]]-y[tris[:,0]]))

# Reorder triangles as necessary so they all have counterclockwise winding order. CCW is what Datashader and MPL
# require.
#
def orderCCW(x,y,tris):
    tris[triArea(x,y,tris)<0.0,:] = tris[triArea(x,y,tris)<0.0,::-1]
    return(tris)

def createHVTriMesh(x,y,triangle_indices, var, var_name, n_workers=1):
    # Declare verts array
    # This is essentally an XYZ matrix (at the location of x and y, z=?) 3 X len(verts) array
    verts = np.column_stack([x, y, var])

    # Convert to pandas
    verts_df  = pd.DataFrame(verts,  columns=['Longitude', 'Latitude', var_name])
    tris_df   = pd.DataFrame(triangle_indices, columns=['v0', 'v1', 'v2'])

    # Convert to dask
    verts_ddf = dd.from_pandas(verts_df, npartitions=n_workers)
    tris_ddf = dd.from_pandas(tris_df, npartitions=n_workers)

    # Declare HoloViews element
    tri_nodes = hv.Nodes(verts_ddf, ['Longitude', 'Latitude', 'index'], [var_name])
    trimesh = hv.TriMesh((tris_ddf, tri_nodes))
    return(trimesh)

# Fetch lat and lon coordinates for the primal and dual mesh.
# Cells are the primary (hexagonal) mesh. 
# lonCell, latCell are the Cartesian coordinates for the centroid of the primal mesh and also
# the vertices of the (triangular) dual mesh.
#lonCell = np.degrees(c_mesh_ds['lonCell'].values) #* 180.0 / math.pi
#latCell = np.degrees(c_mesh_ds['latCell'].values) #* 180.0 / math.pi
# Converts to [-180, 180]
#lonCell = ((lonCell - 180.0) % 360.0) - 180.0
lonCell = np.mod(np.rad2deg(c_mesh_ds['lonCell'].values) - 180.0, 360.0) - 180.0
latCell = np.rad2deg(c_mesh_ds['latCell'].values)

# Vertex are the vertices of the primal (hexagonal) mesh
# lonVertex, latVertex are the Cartesian coordinates for the vertices of the primal mesh and also
# the centroids of the (triangular) dual mesh.
# lonVertex = np.degrees(c_mesh_ds['lonVertex'].values) #* 180.0 / math.pi
# latVertex = np.degrees(c_mesh_ds['latVertex'].values) #* 180.0 / math.pi
# Converts to [-180, 180]
#lonVertex = ((lonVertex - 180.0) % 360.0) - 180.0
lonVertex = np.mod(np.rad2deg(c_mesh_ds['lonVertex'].values) - 180.0, 360.0) - 180.0
latVertex = np.rad2deg(c_mesh_ds['latVertex'].values)

# Get triangle indices for each vertex in the MPAS file. Note, indexing in MPAS starts from 1, not zero :-(
# cellsOnVertex are the hexagonal cells that contain the vertices of the triangular dual mesh.
# cellsOnVertex.values returns the indices for these hexagonal cells, which is based on the centroids of said
# cells, essentially returning the indexes of the vertices for the triangular dual mesh.
# tris = indices for dual mesh vertices
tris = c_mesh_ds.cellsOnVertex.values - 1

# The MPAS connectivity array unforunately does not seem to guarantee consistent counterclockwise winding order, which
# is required by Datashader (and Matplotlib)
# Ironically, this is equal to tris (np.array_equal), but not equal to tris_ccw_flat
tris_ccw = orderCCW(lonCell,latCell,tris)

# Lastly, we need to "unzip" the mesh along a constant line of longitude so that when we project to PCS coordinates
# cells don't wrap around from east to west. The function below does the job, but it assumes that the
# central_longitude from the map projection is 0.0. I.e. it will cut the mesh where longitude
# wraps around from -180.0 to 180.0. We'll need to generalize this
tris_ccw_flat = unzipMesh(lonCell,tris,90.0)

# Project triangle vertices from geographic to PCS coordinates
# Robinson is best globally and has the least deformation in the tropics, but it may be worthwhile to explore others
# Cylindrical projections are best globally and and at the equator (distortion in mid-latitudes)
# Conical projections are best in the mid-high latitudes (20-60 deg)
# Azimuthal projections are best in the poles

sel_time = 23
primalVar = h3cn_ds['FLUT'].isel(time=sel_time).values
lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', sel_time, center_dist=1000)

project_coords = True
if project_coords == True:
    proj = ccrs.Robinson(central_longitude=0.0)
    xPCS, yPCS, _ = proj.transform_points(ccrs.PlateCarree(), lonCell, latCell).T
    trimesh = createHVTriMesh(xPCS,yPCS,tris_ccw_flat, primalVar, 'FLUT', n_workers=1)
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    lon_range = tuple(x_range)
    lat_range = tuple(y_range)

else:
    proj = ccrs.PlateCarree()
    lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
    trimesh = createHVTriMesh(lonCell,latCell,tris_ccw, primalVar, 'FLUT', n_workers=1)

rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, vdim_prefix='', pixel_ratio=5, x_range=lon_range, y_range=lat_range)

mpas_native_olr = rasterized.opts(tools=['hover'], colorbar=True, cmap=flut_cimss, title='MPAS (Native) Upwelling Longwave Flux', 
                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), width=1000, height=900, clabel='Upwelling Longwave Flux [W/m^2]',
                           colorbar_position='bottom', fontsize=dict(title='36pt', clabel='30pt', cticks='28pt')) * gf.coastline(projection=proj).options(scale='10m')


In [ ]:
hv.save(mpas_native_olr, '../figs/mpas_native_olr.png', fmt='png')

In [ ]:
olr_figure = hv.Layout([cam_native_olr, mpas_native_olr]).cols(1)

In [ ]:
hv.save(olr_figure, '../figs/olr_native_left.png', fmt='png')

In [ ]:
#lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', 23, center_dist=1000)
lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
proj = ccrs.Robinson(central_longitude=180.0)
mpas_regridded_olr = h3cr_ds.FLUT.isel(time=23).hvplot('lon', 'lat', projection=proj, project=True, rasterize=True, aggregator='mean', colorbar=False,
                                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), cmap=plt.get_cmap('FLUT_CIMSS'), title='MPAS (Regridded) Upwelling Longwave Flux',
                                           dynamic=False, xlim = lon_range, ylim = lat_range, coastline='10m', fontsize=dict(title='30pt'),
                                           min_width=w, max_width=w, min_height=h, max_height=h)

mpas_regridded_olr

In [ ]:
import hvplot.xarray

lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))

w = 1000
h = 800

mpas_regridded_olr = h3cr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', projection=proj, project=True, rasterize=True, aggregator='mean', colorbar=False,
                                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), cmap='FLUT_CIMSS', title='MPAS (Regridded) Upwelling Longwave Flux',
                                           dynamic=False, xlim = lon_range, ylim = lat_range, coastline='10m', fontsize=dict(title='30pt'),
                                           min_width=w, max_width=w, min_height=h, max_height=h)

cam_regridded_olr = h3pr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', projection=proj, project=True, rasterize=True, aggregator='mean', colorbar=False,
                                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), cmap='FLUT_CIMSS', title='CAM5 (Regridded) Upwelling Longwave Flux ',
                                           dynamic=False, xlim = lon_range, ylim = lat_range, coastline='10m', fontsize=dict(title='30pt'),
                                           min_width=w, max_width=w, min_height=700, max_height=700)

In [ ]:
times = pd.date_range('2008-08-30', '2008-09-04', freq='3H')

#sel_time = 23
primalVar = h3cn_ds['FLUT'].isel(time=sel_time).values
lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', sel_time, center_dist=1000)

for i in range(41):
    primalVar = h3cn_ds['FLUT'].isel(time=i).values
    lon_range, lat_range = find_TC_bbox(h3cn_ds, 'florida', i, center_dist=1000)
    final = rasterized.opts(tools=['hover'], colorbar=True, cmap=flut_cimss, title=f'Upwelling Longwave Flux MPAS (Native Grid) {times[i]}', 
                            clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), width=800, clabel='Upwelling Longwave Flux [W/m^2]', 
                            fontsize=dict(title='16pt', cticks='10pt')) * gf.coastline(projection=proj).options(scale='10m')
    hv.save(final, f'../figs/flut_gif/MPAS_880/{title}.png', backend='bokeh', dpi=300)

## CAM

In [ ]:
def triangulate(vertices, x="Longitude", y="Latitude"):
    """
    Generate a triangular mesh for the given x,y,z vertices, using Delaunay triangulation.
    For large n, typically results in about double the number of triangles as vertices.
    """
    triang = Delaunay(vertices[[x,y]].values)
    print('Given', len(vertices), "vertices, created", len(triang.simplices), 'triangles.')
    
    tris_df = pd.DataFrame(triang.simplices, columns=['v0', 'v1', 'v2'])
    
    return tris_df

def createHVTriMesh(x, y, var, var_name, n_workers=1):
    # Declare verts array
    # This is essentally an XYZ matrix (at the location of x and y, z=?) 3 X len(verts) array
    verts = np.column_stack([x, y, var])

    # Convert to pandas
    verts_df  = pd.DataFrame(verts,  columns=['Longitude', 'Latitude', var_name])
    
    # Creates Delaunay triangular mesh
    tris_df = triangulate(verts_df)

    # Convert to dask
    verts_ddf = dd.from_pandas(verts_df, npartitions=n_workers)
    tris_ddf = dd.from_pandas(tris_df, npartitions=n_workers)

    # Declare HoloViews element
    tri_nodes = hv.Nodes(verts_ddf, ['Longitude', 'Latitude', 'index'], [var_name])
    trimesh = hv.TriMesh((tris_ddf, tri_nodes))
    return(trimesh)

# Grid centerpoints
lonCell = p_mesh_ds.grid_center_lon.values
latCell = p_mesh_ds.grid_center_lat.values
lonCell = ((lonCell - 180.0) % 360.0) - 180.0

# Creates Delaunay triangular mesh
# primalVar = h3pn_ds['FLUT'].isel(time=sel_time).values
# trimesh = createHVTriMesh(lonCell, latCell, primalVar, 'FLUT')

# If you want to project coordinates
# proj = ccrs.Robinson(central_longitude=0.0)
# xPCS, yPCS, _ = projection.transform_points(ccrs.PlateCarree(), lonCell, latCell).T

# If not
# proj = ccrs.PlateCarree()

# lon_range, lat_range = find_TC_bbox(h3pn_ds, 'florida', sel_time, center_dist=1000)
# lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))


sel_time = 23
primalVar = h3pn_ds['FLUT'].isel(time=sel_time).values
lon_range, lat_range = find_TC_bbox(h3pn_ds, 'florida', sel_time, center_dist=1000)

project_coords = True

if project_coords == True:
    proj = ccrs.Robinson(central_longitude=0.0)
    # Creates Delaunay triangular mesh
    xPCS, yPCS, _ = proj.transform_points(ccrs.PlateCarree(), lonCell, latCell).T
    trimesh = createHVTriMesh(xPCS, yPCS, primalVar, 'FLUT')
    x_range, y_range, _ = proj.transform_points(ccrs.PlateCarree(), np.array(lon_range), np.array(lat_range)).T
    lon_range = tuple(x_range)
    lat_range = tuple(y_range)
    
else:
    proj = ccrs.PlateCarree()
    lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
    # Creates Delaunay triangular mesh
    trimesh = createHVTriMesh(lonCell, latCell, primalVar, 'FLUT')


rasterized = hds_rasterize(trimesh, aggregator='mean', precompute=True, x_range=lon_range, y_range=lat_range, vdim_prefix='')

cam_native_olr = rasterized.opts(tools=['hover'], colorbar=False, cmap=flut_cimss, title='CAM5 (Native) Upwelling Longwave Flux', 
                        clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), width=1000, height=800, 
                        fontsize=dict(title='38pt')) * gf.coastline(projection=proj).options(scale='10m')

In [ ]:
hv.save(cam_native_olr, '../figs/cam_native_olr.png', fmt='png')

In [ ]:
# center_dist = 1000 # km
# lon_range, lat_range = find_TC_bbox(h3pn_ds, 'florida', 23, center_dist=center_dist)

lon_range, lat_range = basin_bboxes('florida')
lon_range = tuple(((np.subtract(lon_range, 180) % 360) - 180.0))
lon_range = lon_range

x = p_mesh_ds.grid_center_lon.values
x = ((np.subtract(x, 180) % 360) - 180.0)
y = p_mesh_ds.grid_center_lat.values
var = h3pn_ds.FLUT.isel(time=23).values

points = np.column_stack([x, y, var])
points = points[np.where((points[:, 0] >= lon_range[0]) & (points[:, 0] <= lon_range[1]) & 
                         (points[:, 1] >= lat_range[0]) & (points[:, 1] <= lat_range[1]))]
grid_x, grid_y = np.meshgrid(points[:, 0], points[:, 1], indexing='xy', sparse=False)
z = np.tile(points[:, 2], (len(points[:, 2]), 1))

qmesh = hv.QuadMesh((grid_x, grid_y, z))

In [ ]:
qmesh

In [ ]:
rasterized = hds_rasterize(qmesh, aggregator='mean', precompute=True, x_range=lon_range, y_range=lat_range, vdim_prefix='')

final = rasterized.opts(tools=['hover'], colorbar=True, cmap='viridis', title='Upwelling Longwave Flux CAM (Native Grid)', 
                        width=800, clabel='Upwelling Longwave Flux [W/m^2]', 
                        fontsize=dict(title='16pt', cticks='10pt')) * gf.coastline(projection=proj).options(scale='10m')

final

In [ ]:
p_mesh_ds.grid_center_lat.values.max()

In [ ]:
print(h2pn_ds.lon.values.min())
print(h2pn_ds.lon.values.max())

print(h2pn_ds.lat.values.min())
print(h2pn_ds.lat.values.max())

In [ ]:
def createHVQuadMesh(x, y, corner_indices, var, var_name, n_workers=1):
    """
    x: projected longitude of quad centroids
    y: projected latitude of quad centroids
    quad_indices: endpoints of edges
    var: target variable values
    var_name: target variable name
    """
    
    # Declare verts array
    centers = np.column_stack([x, y, var])

    # Convert to pandas
    centers_df  = pd.DataFrame(centers,  columns=['Longitude', 'Latitude', var_name])
    corners_df   = pd.DataFrame(corner_indices, columns=['v0', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9'])

    # Convert to dask
    centers_ddf = dd.from_pandas(centers_df, npartitions=n_workers)
    corners_ddf = dd.from_pandas(corners_df, npartitions=n_workers)

    # Declare HoloViews element
    quad_nodes = hv.Nodes(centers_ddf, ['Longitude', 'Latitude', 'index'], [var_name])
    quadmesh = hv.QuadMesh((corners_ddf, quad_nodes))
    return(quadmesh)

# Grid centerpoints
lonCell = math.degrees(p_mesh_ds.grid_center_lon.values) #* 180.0 / math.pi
latCell = math.degrees(p_mesh_ds.grid_center_lat.values) #* 180.0 / math.pi
lonCell = ((lonCell - 180.0) % 360.0) - 180.0

# Vertex endpoints
lonVertex = p_mesh_ds.grid_corner_lon.values * 180.0 / math.pi
latVertex = p_mesh_ds.grid_corner_lat.values * 180.0 / math.pi
lonVertex = ((lonCell - 180.0) % 360.0) - 180.0

quads = p_mesh_ds.grid_size.values - 1
#tris_ccw = orderCCW(lonCell,latCell,tris)
#tris_ccw_flat = unzipMesh(lonCell,tris,90.0)

quads = unzipMesh(latVertex,tris,90.0)

projection = ccrs.Robinson(central_longitude=0.0)
xPCS, yPCS, _ = projection.transform_points(ccrs.PlateCarree(), lonCell, latCell).T

primalVar = h3pn_ds['FLUT'].isel(time=23).values
quadmesh = createHVQuadMesh(xPCS,yPCS,quads, primalVar,n_workers=1)

In [ ]:
import datashader
canvas = datashader.Canvas()
datashader.transfer_functions(canvas.quadmesh(h3cn_ds.PSL.isel(time=23).values, 
                                              x=p_mesh_ds.grid_center_lon.values, y=p_mesh_ds.grid_center_lat.values))

In [ ]:
c_mesh_ds.cellsOnVertex.values

In [ ]:
c_mesh_ds

In [ ]:
c_mesh_ds.u10.values

In [ ]:
c_mesh_ds.lonCell.values.max()

In [ ]:
# Trial 3: Using (Curvilinear) Canvas.quadmesh

#hv.QuadMesh

In [ ]:
# Trial 4: Using (Irregular) Triangle Meshes
# I don't think this is appropriate
from holoviews.operation.datashader import rasterize as hds_rasterize


### MPAS

In [ ]:
hv.output(dpi=200, backend='bokeh', fig='png')
central_longitude = 0.0
projection = ccrs.Robinson(central_longitude=central_longitude)

target_var = 'FLUT'
target_time = 23
pixel_ratio = 5
title = f'Upwelling Longwave Flux: {str(h3cn_ds["time"].values[target_time])} UTC'

lon_range, lat_range = find_TC_bbox(h3cn_ds, 'north atlantic zoomed', target_time, center_dist=1000)
rasterized_0831 = datashader_wrapper(c_mesh_ds, h3cn_ds, target_var, target_time, pixel_ratio=pixel_ratio, 
                                     lon_range=lon_range, lat_range=lat_range)
final = rasterized_0831.opts(tools=['hover'], colorbar=True, cmap='FLUT CIMSS', title=title, 
                             clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), width=800) *\
gf.coastline(projection=projection).options(scale='10m')
#hv.save(final, f'../datashader/output_imgs/{title}.png', backend='bokeh', dpi=300)

final

In [ ]:
from matplotlib.gridspec import GridSpec

proj=ccrs.PlateCarree()
projection = ccrs.Robinson(central_longitude=0.0)
fig, ax = plt.subplots(figsize=(10,10), dpi=200, subplot_kw=dict(projection=projection))

lons, lats = basin_bboxes('florida')
ax.set_extent([lons[0], lons[1], lats[0], lats[1]])
ax.coastlines(resolution='10m', linewidth=0.5, edgecolor='#323232')

h3cr_ds['FLUT'].isel(time=23).compute().plot(ax=gax, transform=proj, add_colorbar=False,
                                             cmap='FLUT CIMSS', vmin=T_to_FLUT(-110, 'C'), vmax=T_to_FLUT(55, 'C'))
plt.show()

### CAM5

In [ ]:
proj=ccrs.PlateCarree()
projection = ccrs.Robinson(central_longitude=0.0)
fig, ax = plt.subplots(figsize=(10,10), dpi=200, subplot_kw=dict(projection=projection))

lons, lats = basin_bboxes('florida')
ax.set_extent([lons[0], lons[1], lats[0], lats[1]])
ax.coastlines()
#ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.5, edgecolor='#323232', zorder=10, transform=projection)

h3pr_ds['FLUT'].isel(time=23).compute().plot(ax=ax, transform=proj, cmap='FLUT CIMSS', vmin=T_to_FLUT(-110, 'C'), vmax=T_to_FLUT(55, 'C'))
plt.show()

In [ ]:
h2cr_ds['FLUT'].isel(time=13).compute().plot(cmap='FLUT CIMSS')